In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import networkx as nx

from itertools import tee

In [3]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [4]:
infilepath = r'/mnt/sda1/Users/Work/Temp/ODs_purpose_mode_timeperiod.csv'

In [5]:
odtemp = pd.read_csv(infilepath, nrows=5)

In [6]:
dtypes_keys = list(odtemp.columns)
dtypes_vals = ['category', 'category', 'category',
               'category', 'category', 'float']

dtypes = dict(zip(dtypes_keys, dtypes_vals))

od = pd.read_csv(infilepath, index_col=list(range(0, 5)))
#od.sort_index(inplace=True)

In [7]:
od.head(2)

Trips
orig_zoneID dest_zoneID Purpose Mode TimePeriod           
E02000001   E02000001   HBO     Bus  AM          20.572919
            E02000576   HBO     Bus  AM           3.849908

In [8]:
d_grp_purp = {'HBW': 'HB', 'HBO': 'HB', 'NHB': 'NHB'}
grp_keys = [None, None, d_grp_purp, None, None]

odg = od.groupby(grp_keys, level=od.index.names).sum()

In [9]:
odg.reset_index(inplace=True)
odg.Purpose = odg.Purpose.astype('category')

In [10]:
odg.head(2)

,orig_zoneID,dest_zoneID,Purpose,Mode,TimePeriod,Trips
0,E02000001,E02000001,HB,Bus,AM,93.610267
1,E02000001,E02000001,HB,Bus,IP,189.761532


for c in odg:
    if odg[c].dtype.name == 'category':
        odg[c] = odg[c].cat.codes

In [11]:
odg.set_index(['orig_zoneID', 'dest_zoneID', 'Purpose', 'Mode', 'TimePeriod'], inplace=True)

In [12]:
od_car = odg.xs('CarDriver', level='Mode')

In [13]:
od_car.reset_index(inplace=True)

for c in od_car:
    if od_car[c].dtype.name == 'category':
        od_car[c] = od_car[c].cat.codes

In [31]:
# Reduce
od_car_red = od_car[od_car.Trips >= 20]

In [64]:
G = nx.from_pandas_dataframe(od_car_red, 'orig_zoneID', 'dest_zoneID',
                             ['Purpose', 'TimePeriod'], nx.MultiDiGraph())

In [33]:
G.number_of_nodes()

912

In [34]:
G.number_of_edges()

39208

In [35]:
%time sum(1 for x in nx.trip_chains(G, 5))

CPU times: user 1min 38s, sys: 136 ms, total: 1min 38s
Wall time: 1min 39s


875998

In [36]:
ptc = nx.trip_chains(G, 5)

In [59]:
a = next(ptc)

In [75]:
a[2]

'E02000041'

In [65]:
G.get_edge_data(a[0], a[1])

{0: {'Purpose': 'HB', 'TimePeriod': 'IP'}}

In [66]:
G.get_edge_data(a[1], a[2])

{0: {'Purpose': 'HB', 'TimePeriod': 'AM'},
 1: {'Purpose': 'HB', 'TimePeriod': 'IP'},
 2: {'Purpose': 'HB', 'TimePeriod': 'OP'},
 3: {'Purpose': 'HB', 'TimePeriod': 'PM'},
 4: {'Purpose': 'NHB', 'TimePeriod': 'IP'}}

In [76]:
G.get_edge_data('E02000434', 'E02000041' )

{0: {'Purpose': 'HB', 'TimePeriod': 'AM'},
 1: {'Purpose': 'HB', 'TimePeriod': 'IP'},
 2: {'Purpose': 'HB', 'TimePeriod': 'OP'},
 3: {'Purpose': 'HB', 'TimePeriod': 'PM'},
 4: {'Purpose': 'NHB', 'TimePeriod': 'IP'}}

In [77]:
od_car_red.set_index(['orig_zoneID', 'dest_zoneID']).loc[a[1], a[2]]

Purpose TimePeriod      Trips
orig_zoneID dest_zoneID                              
E02000434   E02000041        HB         AM  42.315216
            E02000041        HB         IP  85.145861
            E02000041        HB         OP  37.984611
            E02000041        HB         PM  42.284094
            E02000041       NHB         IP  20.777397